# Running a 200Mhz 4-Receptor Correlation Flow
###### Last Updated: 15/04/24

This notebook will run through running through a 200Mhz receptor correlation, using 4 Talon boards.

## 1 Prerequisites

Before running this notebook, ensure you have: 
* A namespace running from the [SKA-mid-psi](https://gitlab.com/ska-telescope/ska-mid-psi) pipeline deploy step, or a compatible one.
* A virtual env with [poetry](https://python-poetry.org/docs/basic-usage/#installing-dependencies) run on it to ensure all requirements are installed.
* Python 3.10 and the above venv selected as the interpreter for the notebook. 
* Four boards (1,2,3,4 or 5,6,7,8) checked out, unless you plan to run in simulation mode.

We can then start this notebook by importing the libraries we will need.

In [ ]:
import json
import os
from time import sleep

from tango import Database, DevFailed, DeviceProxy

As we will need to wait for a number of stages to be reached by the devices, we use this to wait for device state

In [ ]:
spinner = ["⣾", "⣽", "⣻", "⢿", "⡿", "⣟", "⣯", "⣷"]


def wait_for_state(device: DeviceProxy, desired_state, break_on_error=True) -> None:
    """Poll a tango device until either the given observation state is reached, or it throws an error.
    Arguments:
    device -- Tango Device to check
    desired_state -- The state which to break upon getting (number or state)
    break_on_error -- If set to False, will keeping running when getting an error status.
    """
    spinL = 0
    poll = 1
    while device.obsState != desired_state:
        if spinL < len(spinner) - 1:
            spinL += 1
        else:
            spinL = 0
        sleep(0.5)
        print(
            "\r",
            f"{spinner[spinL]} Poll# {poll}: Current state is {device.obsState.name}, waiting for {desired_state}...",
            end="",
        )
        if device.obsState == 9 and break_on_error:
            break
        poll += 1
    print(f"\nFinished with: {device.obsState.name}")

### 1.1 Running on MID PSI

After the deploy namespace job has been executed in the GitLab pipeline, retrieve your namespace by running the below block and grabbing the namespace with your name. Use this to set the `ns` variable in the python block below.

In [ ]:
!kubectl get ns | grep ska-mid-psi

### 1.2 Loading in Variables/Checking Taranta

Now, we set the variables we'll need for the run. Ensure that `ns`, `target_boards_list` and `simulation_mode` are set as expected, and that you provide the correct config files.

In [ ]:
# --Required Vars--
# Non-SPD namespace
ns = ""  # UPDATE THIS FOR RUN
# The boards we will be using for the test. Should only use either 1,2,3,4 or 5,6,7,8 to prevent issues with assignment.
target_boards_list = []  # UPDATE THIS FOR RUN
# Set to 0 for off, 1 for on
simulation_mode = 0  # UPDATE THIS FOR RUN

# Slim receptor config, leave blank if using default (1 board)
slim_fs_config = "fs_slim_4vcc_1fsp.yaml"
slim_vis_config = ""

# Delay Model file, set to alt if using one board
delay_model_filename = "delay_model_4r.json"

With `ns` now set, you should be able to check that pods are correctly deployed to it.

In [ ]:
!kubectl -n $ns get pods

We can also set the rest of the vars we need. These should not need to be changed.

In [ ]:
# MAC address for the boards.
TARGET_MAC_ADDRESS = "08:c0:eb:9d:47:78"
# Parent directory to use to grab config files.
DATA_DIR = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "data")
# Config file directories
COMMON_CONFIG = os.path.join(DATA_DIR, "mid_telescope/cbf")
CSP_CONFIG = os.path.join(DATA_DIR, "mid_telescope/csp")
TMC_CONFIG = os.path.join(DATA_DIR, "mid_telescope/tmc")
HW_CONFIG = os.path.join(COMMON_CONFIG, "hw_config")
SLIM_CONFIG = os.path.join(COMMON_CONFIG, "slim_config")
# For mapping the talon boards to receptor
RECEPTOR_MAP = ["SKA001", "SKA036", "SKA063", "SKA100"]
TARGET_BOARDS_STR = ",".join(str(x) for x in target_boards_list)
BITE_MODE = ""

With these set, we can now check the taranta dash to monitor the boards as we run through the rest of the notebook.

In [ ]:
url = "https://142.73.34.170/" + ns + "/taranta/dashboard?id=65e7b6f7b72ec70018cdb16a&mode=run"
print(
    "You can monitor board status using: https://142.73.34.170/{}/taranta/dashboard?id=660de6afb20f1600120ec597&mode=run".format(
        ns
    )
)

At this point, all components should be in the `disabled` state.

The `TANGO_HOST` environment variable will be created based of the namespace you set earlier, and allows us to communicate to the TANGO devices.

In [ ]:
TANGO_HOST = "databaseds-tango-base." + ns + ".svc.cluster.local:10000"
print("Will be using HOST: ", TANGO_HOST)
os.environ["TANGO_HOST"] = TANGO_HOST

### 1.3 Loading Config Files

We then load in the locations of local JSON files for configuration:

In [ ]:
print("Getting files...")

INIT_SYS_PARAM_FILE = os.path.join(COMMON_CONFIG, "sys_params/initial_system_param.json")
ASSIGN_RESOURCES_FILE = os.path.join(TMC_CONFIG, "assign_resources.json")
ASSIGN_CSP_RESOURCES_FILE = os.path.join(CSP_CONFIG, "assign_resources.json")
CONFIGURE_SCAN_FILE = os.path.join(TMC_CONFIG, "configure_scan.json")
SCAN_FILE = os.path.join(TMC_CONFIG, "scan.json")
CSP_DELAY_MODEL_FILE = os.path.join(TMC_CONFIG, delay_model_filename)
SLIM_CONFIG_FILE = os.path.join(SLIM_CONFIG, slim_fs_config)

files = [
    INIT_SYS_PARAM_FILE,
    ASSIGN_RESOURCES_FILE,
    ASSIGN_CSP_RESOURCES_FILE,
    ASSIGN_CSP_RESOURCES_FILE,
    CONFIGURE_SCAN_FILE,
    SCAN_FILE,
    CSP_DELAY_MODEL_FILE,
    SLIM_CONFIG_FILE,
]

for file in files:
    if os.path.isfile(file):
        print(f"{file} exists: ✔️")
    else:
        print(f"{file} does not exist ❌")

print("Done")

Next, we pass in the SLIM mesh config files by copying them to the namespace, these files should be in the json_files storage folder. Custom files here are not required, but if needed, the following two code block can be used.

In [ ]:
if slim_fs_config != "":
    print("Loading custom SLIM fs config")
    !kubectl cp $SLIM_CONFIG_FILE $ns/ds-cbfcontroller-controller-0:/app/mnt/slim/fs_slim_config.yaml
else:
    print("SLIM fs will use defaults for this test.")
if slim_vis_config != "":
    print("Loading custom SLIM vis config")
    !kubectl cp $SLIM_CONFIG_FILE $ns/ds-cbfcontroller-controller-0:/app/mnt/slim/vis_slim_config.yaml
else:
    print("SLIM vis will use default config")

We then load in the hardware configuration depending on the talon boards selected. If the higher number boards are used, we need to use the swapped config file, and then modify each board value to match the swap file.

**DO NOT mix higher number boards with lower number ones (2,3,4,5), as this will cause issues with the hardware config**

In [ ]:
if any(i > 4 for i in target_boards_list):
    print("Using swap for higher number talons")
    config = "hw_config_swap_psi.yaml"
    print("Modifying target to use lower nums to match swap file")
    target_boards_list = list(map(lambda x: x - 4, target_boards_list))

else:
    print("Using standard HW config")
    config = "hw_config_psi.yaml"

HW_CONFIG_FILE = os.path.join(HW_CONFIG, config)
if os.path.isfile(HW_CONFIG_FILE):
    print("HW config: ✔️")
else:
    print("hw config: ❌")
TARGET_BOARDS_STR = ",".join(str(x) for x in target_boards_list)

## 2 Initial Setup

For this demo, we will interact with the TANGO devices via a device proxy, which will allow us to pass commands into them as we would in the UI.

In [ ]:
# Setup the device proxies targeting bite and deployer
db = Database()

deployer = DeviceProxy("mid_csp_cbf/ec/deployer")
# Check the devices initially deployed to the database
print("Currently exported devices:")
print(*db.get_device_exported("*").value_string, sep="\n")
# Make sure the deployer device is set to ON
deployer.On()
print(deployer.state())

Now we copy this file into the controller pod:

In [ ]:
!kubectl cp $HW_CONFIG_FILE $ns/ds-cbfcontroller-controller-0:/app/mnt/hw_config/hw_config.yaml

## 3 Deploying

### 3.1 Deploying Using the Command Line

To use the old method for generating the talon config, we use kubectl's exec function to run commands on the relevant pod, using the deployer script and passing the relevant commands to it as arguments.

In [ ]:
print("Targeting boards: {}".format(TARGET_BOARDS_STR))
!kubectl exec -ti -n $ns ec-deployer -- python3 midcbf_deployer.py --generate-talondx-config --boards=$TARGET_BOARDS_STR

Once it completes, we can run the deployer device download and configure commands as normal.

### 3.2 Deploying Using the Deployer Device

First, we set the target talon boards we want to set up our configuration for. In Jive/Taranta, this would be configured by manually writing the attribute via the UI. Multiple boards can be targeted. With this variable set on the device, we can then run the configuration command by calling the generate_config_jsons command.

In [ ]:
deployer.targetTalons = target_boards_list
print(deployer.targetTalons)
deployer.generate_config_jsons()

We then get the device artifacts from the [artifact repository](https://artefact.skao.int/#browse/browse:helm-internal) by running the command via TANGO. This step may take some time as it downloads multiple devices.

In [ ]:
deployer.set_timeout_millis(400000)
try:
    deployer.download_artifacts()
except DevFailed as e:
    print(e)
    print(
        "Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes."
    )
deployer.set_timeout_millis(3000)

To check that the artifacts downloaded successfully, we want to check that the following returns something like `INFO|Dummy-2|download_fpga_bitstreams|midcbf_deployer.py#418||Finished downloading`. 

In [ ]:
!kubectl logs -n $ns ds-deployer-deployer-0 | grep 'Finished downloading'

Finally, we can configure the TANGO database with all the tango devices we just downloaded using the ConfigDB command. It should now be configured with all the devices needed for the next step, running the BITE device

In [ ]:
deployer.configure_db()
print("Currently exported devices:")
print(*db.get_device_exported("*").value_string, sep="\n")

## 4 Uploading Controller Settings and Starting the Boards

Now that we have the requried devices deployed and exported, we can set up the DeviceProxies for the devices we'll use in the next steps:

In [ ]:
controller = DeviceProxy("mid-csp/control/0")
print(controller.status())
subarray = DeviceProxy("mid-csp/subarray/01")
print(subarray.status())
cbf = DeviceProxy("mid_csp_cbf/sub_elt/controller")
print(cbf.status())

**Before running this step and the following steps, ensure that you have checked out the boards you have selected to use!**
***

We can then set adminMode to 0 (ONLINE), allowing us to run commands, and set simulationMode to 0 (FALSE).

In [ ]:
# Set relevant values on the mid-csp controller
controller.adminMode = 0
sleep(1)
controller.write_attribute("cbfSimulationMode", simulation_mode)
controller.cbfSimulationMode = simulation_mode
sleep(1)

if (
    controller.read_attribute("adminMode").value == 0
    and controller.read_attribute("cbfSimulationMode").value == 0
):
    print("Set to simulation mode off.")
elif (
    controller.read_attribute("adminMode").value == 0
    and controller.read_attribute("cbfSimulationMode").value == 1
):
    print("Set to simulation mode on.")
else:
    print("Error, couldn't set values!")

Checking the status dashboard, it should now display that all devices are both OFF and that the simulationstate is FALSE.

 Next, we load in a inital values parameters and pass it to the controller, to do this we read in JSON file and pass it as a DevString to the relevant device command: 

In [ ]:
with open(INIT_SYS_PARAM_FILE, encoding="utf-8") as init_file:
    data = json.load(init_file)
print("Initial system parameter file:")
print(json.dumps(data, indent=1))

In [ ]:
# After confirming the file is correct load into the CBF
upload_result = cbf.InitSysParam(json.dumps(data))

Now, we turn ON the Controller by passing it the device we want to turn on, and letting it run for 100s to give the boards time to power on. Before running this step and further ones, consider using [k9s](https://k9scli.io/) in a separate shell to monitor the CBF controller device.

In [ ]:
print(controller.status())
controller.set_timeout_millis(100000)
target = ["mid_csp_cbf/sub_elt/controller"]
controller.On(target)
if simulation_mode == 1:
    sleep(5)
else:
    sleep(100)
print(controller.status())

After running this step, check with the Taranta dashboard to check that the boards are started.

Additionally, for each board, ssh into each of the boards (`ssh root@talon#`), and run the following commands to ensure that the device servers on each are running.

## 5 BITE Commands

### 5.1 Running Commands in BITE through Command Line Arguments (Old Method)

This is the older method for running BITE commands, utilizing kubectl to run python files directly:

First, we copy in required files for multi-board LSTV generation:

In [ ]:
CBF_TARGET = "/app/images/ska-mid-cbf-engineering-console-bite/test_parameters"
CBF_BASE = "/app/images/ska-mid-cbf-engineering-console-bite"
TEST_DATA = "test_parameters"
print(os.path.isdir(TEST_DATA))

In [ ]:
!kubectl exec ec-bite -n $ns -- rm -rf $CBF_TARGET
!kubectl cp $TEST_DATA $ns/ec-bite:$CBF_BASE

First we configure, specifying the MAC address of the boards and the boards to configure.

In [ ]:
!kubectl exec -n $ns ec-bite -- python3 midcbf_bite.py --talon-bite-config --boards=$TARGET_BOARDS_STR --bite_mac_address=$TARGET_MAC_ADDRESS --input_data=$TARGET_BOARDS_STR
BITE_MODE = "cmd"

### 5.2 Running Commands through the BITE Device Server

Now that the BITE TANGO device server has been deployed via the deployer, we can use it to configure tests. First we check the device is running.

In [ ]:
bite = DeviceProxy("mid_csp_cbf/ec/bite")
# Running this should return RUNNING
print(bite.State())

# Set required vars on BITE device, the boards we want to target and the MAC address for them.
bite.boards = target_boards_list
bite.bite_mac_address = TARGET_MAC_ADDRESS

For now, we can use the defaults and simply call the write command for the test configs. This should return the configuration for each board passed in.

In [ ]:
bite.generate_bite_data()
BITE_MODE = "device"

## 6 Assigning Resources

Next, we use the relevant subarray device to assign resources. First, as with the other devices, we establish a DeviceProxy to connect to it. We also read in the assign_resources file to load in using the command. 

For the loaded data, we must ensure the right boards are selected, modifying the data for the receptor ID based on the Talon board selected.

In [ ]:
# Load in the data from the file.
subarray = DeviceProxy("mid-csp/subarray/01")
with open(ASSIGN_CSP_RESOURCES_FILE, encoding="utf-8") as init_file:
    config_dict = json.load(init_file)

# In order to use the correct receptor, we modify the assign_resources data to use the correct receptor based on the board we're using
config_dict["dish"]["receptor_ids"] = list(map(lambda x: RECEPTOR_MAP[x - 1], target_boards_list))
print(json.dumps(config_dict))

Next, we run the actual command to pass in the resources:

In [ ]:
subarray.AssignResources(json.dumps(config_dict))
wait_for_state(subarray, 2)

## 7 SDP Setup

Next, we run a few commands to set up the Science Data Processor (SDP), utilizing the sdp namespace

In [ ]:
# Set up SDP TANGO device proxy
sdp = DeviceProxy("test-sdp/subarray/01")
sdp.On()
print(sdp.Status())

Now, we assign resources as we did with the subarray, as with the csp, we map the receptor boards based on our checked out board.

In [ ]:
with open(ASSIGN_RESOURCES_FILE, encoding="utf-8") as f:
    resources_json = json.load(f)
resources_json["dish"]["receptor_ids"] = list(
    map(lambda x: RECEPTOR_MAP[x - 1], target_boards_list)
)
resources_json["sdp"]["resources"]["receptors"] = list(
    map(lambda x: RECEPTOR_MAP[x - 1], target_boards_list)
)

sdp_only_json = resources_json["sdp"]
print("SDP resources JSON:")
print(json.dumps(sdp_only_json, indent=1))
print("================================")

In [ ]:
# After confirming the resource assignment is correct, run the actual argument and wait for device to go to IDLE
sdp.AssignResources(json.dumps(sdp_only_json))
wait_for_state(sdp, 2)

Next we send the configure command along with the required config file, making sure we are out of the 'RESOURCING' state.

In [ ]:
with open(CONFIGURE_SCAN_FILE, encoding="utf-8") as f:
    sdp_configuration = json.load(f)
    sdp_configuration = sdp_configuration["sdp"]
print("SDP config JSON:")
print(json.dumps(sdp_configuration))
print("========================")

In [ ]:
# Pass the JSON configuration to the device after ensuring it is correct, waiting to ensure it goes to READY
sdp.Configure(json.dumps(sdp_configuration))
wait_for_state(sdp, 4)

We then start the SDP output via the Scan command:

In [ ]:
with open(SCAN_FILE, encoding="utf-8") as f:
    sdp_scan = json.load(f)
    sdp_scan = sdp_scan["sdp"]
print("Scan JSON:")
print(json.dumps(sdp_scan))
print("========================")

In [ ]:
# Run the SDP's scan command after confirming the scan file is as expected
sdp.Scan(json.dumps(sdp_scan))
print(sdp.obsState)

After running this step, a pod for receiving visibilities will be launched, grab the IP of this pod and place it in the output_host var in the configure_scan json file.

In [ ]:
# Grab the pod name
!kubectl -n $ns-sdp get pods | grep vis-receive

In [ ]:
# Then find the IP:
vis_pod = "proc-pb-test-20211111-00059-vis-receive-00-0"
!kubectl -n $ns-sdp describe pod $vis_pod | grep net1 -A 3

Grab the IP from this, and write it to a var we'll use later to configure the CSP.

In [ ]:
output_host = "10.50.1.34"

With this pod, we will also want to prep for monitoring the pod for when it receives the visibilities

Run this command, then use the output in a separate terminal to enter the correct pod:

In [ ]:
!echo kubectl exec -n $ns-sdp -ti $vis_pod -- bash

We can then run the following commands in this pod to start monitoring for the correct traffic:

## 8 Starting up the CSP Scan

Now on the CSP side we can load in the corresponding Scan and cofig files, modifying the config to prep for our test.

In [ ]:
with open(CONFIGURE_SCAN_FILE, encoding="utf-8") as f:
    csp_configure_scan = json.load(f)["csp"]
with open(SCAN_FILE, encoding="utf-8") as f:
    csp_scan = json.load(f)["csp"]
csp_configure_scan["common"]["config_id"] = "4 receptor, band 1, 1 FSP, no options"
csp_configure_scan["subarray"]["subarray_name"] = "4 receptors"
# Write FSP and related data
csp_configure_scan["cbf"]["fsp"][0]["fsp_id"] = 1
csp_configure_scan["cbf"]["fsp"][0]["zoom_factor"] = 1
csp_configure_scan["cbf"]["fsp"][0]["zoom_window_tuning"] = 450000
csp_configure_scan["cbf"]["fsp"][0]["channel_offset"] = 0
csp_configure_scan["cbf"]["fsp"][0]["receptors"] = list(
    map(lambda x: RECEPTOR_MAP[x - 1], target_boards_list)
)
csp_configure_scan["cbf"]["fsp"][0]["output_host"][0][0] = 0
csp_configure_scan["cbf"]["fsp"][0]["output_host"][0][1] = output_host

csp_configure_scan["cbf"]["fsp"][0]["output_port"][0][0] = 0
csp_configure_scan["cbf"]["fsp"][0]["output_port"][0][1] = 21000
csp_configure_scan["cbf"]["fsp"][0]["output_port"][0][2] = 1

print("Modified CSP scan configuration is:")
print(json.dumps(csp_configure_scan, indent=1))
print("========================")
print("CSP scan file is:")
print(json.dumps(csp_scan, indent=1))
print("========================")

In [ ]:
# After confirming the configure scan file is expected, pass to the csp subarray configure command
subarray.Configure(json.dumps(csp_configure_scan))
sleep(5)
print("CBF subarray Observation state after running Configure: {}".format(subarray.obsState))

Then we start LSTV replay, check the logs and get the epoch value to use later (`INFO: start_utc_time_offset = start_utc_time.unix_tai - ska_epoch_tai = <EPOCH VALUE TO COPY>`)

In [ ]:
if BITE_MODE == "device":
    print("Using the bite device to start lstv replay...")
    bite.start_lstv_replay()
elif BITE_MODE == "cmd":
    !kubectl exec -ti -n $ns ec-bite -- python3 midcbf_bite.py --talon-bite-lstv-replay --boards=$TARGET_BOARDS_STR --input_data=$TARGET_BOARDS_STR
else:
    print("Did you run the configure BITE commands?")

We then store this epoch value to configure the delay model.

In [ ]:
target_epoch = 768090142.0

We then load this into the delay model, and change the `start_validity_sec` value to the one generated by the BITE LSTV replay start command.

In [ ]:
with open(CSP_DELAY_MODEL_FILE, encoding="utf-8") as f:
    delay_model = json.load(f)
delay_model["start_validity_sec"] = target_epoch
print("Delay model JSON:")
print(json.dumps(delay_model, indent=1))
print("========================")

We can then set the delay model we'll be using for the scan, and run the scan on the csp subarray.

In [ ]:
delayModelProxy = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray_01")
delayModelProxy.write_attribute("delayModel", json.dumps(delay_model))
subarray.Scan(json.dumps(csp_scan))
print("Observation state: {}".format(subarray.obsState.name))

Checking the dashboard, the dish should now be in the SCANNING status.

## 9 Checking Visibilities

With the devices scanning and linked, we can monitor the output via monitoring the network packets that come from it. Use the tcpdump running terminal to check that packet lengths are correct:
- UDP, length 136.
- UDP, length 7040.

Also check that the visibility pod is writing data. This can be achieved by using k9s to get the logs from receiver pod in your ns, and checking to see that the following logs appear: `Written data for # vis0 fsp_1_channels to row #`.

## 10 Cleanup

Once we're satisfied with the results, we can stop the scans and shut down the boards.

In [ ]:
# End scanning on the SDP
sdp.EndScan()

In [ ]:
sdp.End()

In [ ]:
# End scanning on the CSP
subarray.EndScan()

In [ ]:
subarray.End()

In [ ]:
# Have the CSP subarray go to the IDLE state and have it release all resources assigned to it.
subarray.GoToIdle()
subarray.ReleaseAllResources()

In [ ]:
# Turn the controller off.
controller.Off(target)

 If you need to, ensure you save any logs you might need, either using k9s or by piping the log output from kubectl log commands to a file. Once you are done using this notebook and the logs, free up dev resources on MID-PSI by deleting your ns.

In [ ]:
!kubectl delete ns $ns
!kubectl delete ns $ns-sdp

In [ ]:
!kubectl get ns | grep ska-mid-psi

🎉 Congrats, you've now run the Auto Correlator demo!